In [ ]:
import numpy as np
from ucimlrepo import fetch_ucirepo
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.kernel_approximation import RBFSampler
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score, classification_report, roc_curve
from sklearn.metrics import roc_auc_score, log_loss, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns


# Fetch dataset
breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17)

# Data (as pandas dataframes)
X = breast_cancer_wisconsin_diagnostic.data.features
y = breast_cancer_wisconsin_diagnostic.data.targets

# Standardizing the data
scaler = StandardScaler()
X = scaler.fit_transform(X)
class_mapping = {'M': 1, 'B': 0}
y = [class_mapping[target] for target in y.T.values[0]]

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

max_components = min(X_test.shape[0], X_test.shape[1])
print("Maximum number of components:", max_components)

num_components= [10, 15, 20]
for every in num_components:
    # Apply PCA for dimensionality reduction (optional)
    # You can skip this step or adjust the number of components based on your needs
    # num_components = 15  # You can choose the number of components you want
    pca = PCA(n_components=every)
    # Fit and transform the training data
    X_train_transformed = pca.fit_transform(X_train)
    # Transform the test data
    X_test_transformed = pca.transform(X_test)
    # Visualize the first two principal components
    explained_var_ratio = pca.explained_variance_ratio_
    # Scree plot
    plt.figure(figsize=(10, 6))
    plt.bar(range(1, every + 1), explained_var_ratio, alpha=0.7, align='center')
    plt.title('Explained Variance Ratio for Each Principal Component')
    plt.xlabel('Principal Component')
    plt.ylabel('Explained Variance Ratio')
    plt.show()

    # Define the model
    model = KNeighborsClassifier()

    # Define the hyperparameter grid
    param_grid = {
        'n_neighbors': [2,3,4,5,6,7,8],
        'weights': ['uniform','distance'],
        'p': [1,2]
    }

    # Create GridSearchCV object with cross-validation
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train_transformed, y_train)

    # Get the best parameters and the corresponding model
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_

    all_models = grid_search.cv_results_['params']

    # Use cross_val_score to get cross-validation scores for each model
    cross_val_scores = []
    for model_params in all_models:
        model = KNeighborsClassifier(**model_params)
        scores = cross_val_score(model, X_train_transformed, y_train, cv=5, scoring='accuracy')
        cross_val_scores.append(scores)

    plt.figure(figsize=(12, 8))
    sns.boxplot(data=cross_val_scores)
    plt.xticks(ticks=range(len(all_models)))

    # Simulate partial_fit with batch-wise training
    batch_size = X_train.shape[0]
    num_samples = X_train.shape[0]
    num_batches = int(np.ceil(num_samples / batch_size))
    print("Best model : ", best_params)

    # best_sgd_model = SGDClassifier(**best_params)
    best_knn_model = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'])

        # Train the model
    best_knn_model.fit(X_train_transformed, y_train)

         # Evaluate the best model on the test set
    y_pred = best_knn_model.predict(X_test_transformed)
    y_prob = best_knn_model.predict_proba(X_test_transformed)

    # Testing accuracy
    test_accuracy = accuracy_score(y_test, y_pred)
    print("Testing Accuracy:", test_accuracy)

    # Classification report
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

    # Confusion matrix
    confusion_mat = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix:")
    print(confusion_mat)
